<a href="https://colab.research.google.com/github/tina-seb/Hackathon2020/blob/master/CDL_Hackathon_2020_JediMasters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Food Bank Delivery Route Scheduler**

**Problem** : Food banks use volunteers on a daily basis to deliver food and groceries to the homebound. Currently they depend on manual scheduling and/or classical software to schedule routes for a variable number of volunteers, which may change during the day.

**Challenges** : The route maps need to be generated quickly if a volunteer drops off, and also need to be the most cost-efficient and fastest route possible.

**Proposed Solution** : We implement a variation of the Multiple Travelling Salesman algorithm using DWave Leap Hybrid Solver and QBSolve to generate the most efficient routes for a variable number of volunteers, by minimizing the cost as energy and total mileage.

**Constraints** : 
1) All volunteeers must start and end at the same food bank location.
2) Each volunteer should only visit one home for delivery.
3) If a volunteer does not show up or cancels, another volunteer should be able to pick up his or her route as an extension, without having to reschedule every other route.
4) The cost of each individual route as well as the cost of the entire schedule should be minimized.

In [ ]:
from dwave_qbsolv import QBSolv
from dwave.system import LeapHybridSampler
import numpy as np
import matplotlib.pyplot as plt
import re

The number of houses with homebound clients served by a food bank changes every day. This is currently defined as a variable and can be changed into an input variable. The home addresses and the distance information of each address needs to be provided as input files to the algorithm.

In [ ]:
# Number of houses in our food delivery problem
Total_Number_Houses = 48
Number_Volunteers = 8
Number_Routes = (int)(Total_Number_Houses/Number_Volunteers)

# Tunable parameters. 
A = 8500
B = 1
chainstrength = 4500
numruns = 100

We initialize and construct a distance matrix where the row and column represent a house address and each entry is a distance in metres.

In [ ]:
## Constructing the distance matrix
D = [[0 for z in range(Total_Number_Houses)] for y in range(Total_Number_Houses)]

# Input file containing inter-house distances for all counties
fn = "FoodBankClientDeliveryData.txt"

# check that the user has provided input file
try:
  with open(fn, "r") as myfile:
    distance_text = myfile.readlines()
    myfile.close()
except IOError:
  print("Input inter-house distance file missing")
  exit(1)

# Extract the distances from the input file. 
for i in distance_text:
  if re.search("^between", i):
    m = re.search("^between_(\d+)_(\d+) = (\d+)", i)
    housea = int(m.group(1))
    houseb = int(m.group(2))
    D[housea][houseb] = D[houseb][housea] = int(m.group(3))


We then construct a QUBO (Quadratic Unconstrained Binary Optimization) to feed into the various DWave samplers. We define the objective and constraints based on our preidentified requirements here.

In [ ]:

# Function to compute index in QUBO for variable 
def return_QUBO_Index(a, b):
    return (a)*Total_Number_Houses+(b)

## Creating the QUBO
# Start with an empty QUBO
Q = {}
for i in range(Total_Number_Houses*Total_Number_Houses):
    for j in range(Total_Number_Houses*Total_Number_Houses):
        Q.update({(i,j): 0})

# Constraint that each row has exactly one 1, constant = N*A
for v in range(Total_Number_Houses):
    for j in range(Total_Number_Houses):
        Q[(return_QUBO_Index(v,j), return_QUBO_Index(v,j))] += -1*A
        for k in range(j+1, Total_Number_Houses):
            Q[(return_QUBO_Index(v,j), return_QUBO_Index(v,k))] += 2*A
            Q[(return_QUBO_Index(v,k), return_QUBO_Index(v,j))] += 2*A

# Constraint that each col has exactly one 1
for j in range(Total_Number_Houses):
    for v in range(Total_Number_Houses):
        Q[(return_QUBO_Index(v,j), return_QUBO_Index(v,j))] += -1*A
        for w in range(v+1,Total_Number_Houses):
            Q[(return_QUBO_Index(v,j), return_QUBO_Index(w,j))] += 2*A
            Q[(return_QUBO_Index(w,j), return_QUBO_Index(v,j))] += 2*A

# Objective that minimizes distance
for u in range(Total_Number_Houses):
    for v in range(Total_Number_Houses):
        if u!=v:
            for j in range(Total_Number_Houses):
                Q[(return_QUBO_Index(u,j), return_QUBO_Index(v,(j+1)%Total_Number_Houses))] += B*D[u][v]


We have two approaches below : A classical solver approach using QBSolv and a Quantum solver approach using Leap Hybrid Sampler. No significant timing differences were found between the two solutions.

In [ ]:
# Run the QUBO using qbsolv (classically solving)
#resp = QBSolv().sample_qubo(Q)

# Use LeapHybridSampler() for faster QPU access
sampler = LeapHybridSampler()
resp = sampler.sample_qubo(Q)

# First solution is the lowest energy solution found
sample = next(iter(resp))

Display the results obtained as a schedule of route maps for the required number of routes.

In [ ]:
# Display energy for best solution found
print('Energy: ', next(iter(resp.data())).energy)

# Print route for solution found
route = [-1]*Total_Number_Houses
for node in sample:
    if sample[node]>0:
        j = node%Total_Number_Houses
        v = (node-j)/Total_Number_Houses
        route[j] = int(v)

# Compute and display total mileage
mileage = 0
for i in range(Total_Number_Houses):
    mileage+=D[route[i]][route[(i+1)%Total_Number_Houses]]

print('Mileage: ', mileage)

# Print route:

houses = [',']*Total_Number_Houses
cities = [',']*Total_Number_Houses

with open('FoodBankHouseLookup.txt', "r") as myfile:
    address_text = myfile.readlines()
    myfile.close()

for i in address_text:
    index, city, house = i.split(',')
    cities[int(index)] = city.rstrip()
    houses[int(index)] = house.rstrip()

output = open('Food_Bank_Delivery.route.offline', 'w')

r_no = 1
for i in range(Total_Number_Houses):
    if(i % Number_Routes == 0):
        output.write('\n' + 'Route ' + str(r_no) + '\n')
        r_no = r_no + 1
    output.write(houses[route[i]] + ',' + cities[route[i]] + '\n')



The route schedule output is as follows :


Route 1
9980 Brookside Dr.,Columbus OH
417 Theatre Lane,Columbus OH
26 Walnut Drive,Columbus OH
7085 Myers St.,Columbus OH
821 Garfield Rd.,Columbus OH
7297 North Park Road,Columbus OH

Route 2
16 Park Ave.,Columbus OH
51 Lexington Dr.,Columbus OH
31 York Drive,Columbus OH
349 W. Rockaway Drive,Columbus OH
19 Center Street,Columbus OH
7750 John Drive,Columbus OH

Route 3
11 High Noon St.,Columbus OH
69 Buckingham St.,Columbus OH
8891 Vine Street,Columbus OH
977 Elizabeth Lane,Columbus OH
78 High Point St.,Columbus OH
44 NW. San Pablo St.,Columbus OH

Route 4
82 Paris Hill St.,Columbus OH
23 S. Woodside Road,Columbus OH
7655 Catherine Road,Columbus OH
70 Coffee Ave.,Columbus OH
7152 Augusta St.,Columbus OH
5 South Greystone Street,Columbus OH

Route 5
192 West Albany St.,Columbus OH
7 Marvon Road,Columbus OH
92 South Country Club Rd.,Columbus OH
98 Galvin Street,Columbus OH
3 Hillcrest Court,Columbus OH
9070 Ramblewood Ave.,Columbus OH

Route 6
8043 N. Glen Ridge Dr.,Columbus OH
7678 Clinton Road,Columbus OH
685 Oakland Court,Columbus OH
737 Pacific Street,Columbus OH
32 Southampton Street,Columbus OH
9697 Bowman Drive,Columbus OH

Route 7
9304 Brickyard St.,Columbus OH
8858 Canal Lane,Columbus OH
8 Old Jackson Avenue,Columbus OH
51 Columbia Street,Columbus OH
1961 Cleveland Ave,Columbus OH
7798 Brown Drive,Columbus OH

Route 8
3432 Main St,Columbus OH
23 Oak Meadow Drive,Columbus OH
287 Military Dr.,Columbus OH
1944 Frank Dr,Columbus OH
64 Illinois Lane,Columbus OH
953 South Fieldstone Drive,Columbus OH
